<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [7]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [9]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# Define the URL for the Falcon 9 Launch Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request and assign the response to an object
response = requests.get(static_url)

# Check the response status code
print(f"Response Status Code: {response.status_code}")

# Optionally, print the content of the page (or part of it) to verify the request
print(response.text[:500])  # Print the first 500 characters of the HTML content


Response Status Code: 200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Create a `BeautifulSoup` object from the HTML `response`


In [13]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup

# Define the URL for the Falcon 9 Launch Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request to fetch the page content
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Use BeautifulSoup() to create a BeautifulSoup object from the response text content
    soup = BeautifulSoup(response.text, 'html.parser')  # You can also use 'lxml' if installed
    
    # Optionally, print the parsed HTML (or part of it)
    print(soup.prettify()[:500])  # Print the first 500 characters of the formatted HTML
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [15]:
# Use soup.title attribute
if response.status_code == 200:
    # Use BeautifulSoup() to create a BeautifulSoup object from the response text content
    soup = BeautifulSoup(response.text, 'html.parser')  # You can also use 'lxml' if installed
    
    # Print the page title to verify if BeautifulSoup object was created properly
    page_title = soup.title.string  # This extracts the title text
    print(f"Page Title: {page_title}")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [17]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Check if the request was successful
if response.status_code == 200:
    # Use BeautifulSoup() to create a BeautifulSoup object from the response text content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all the tables on the page
    tables = soup.find_all('table', {'class': 'wikitable'})  # Tables with class 'wikitable' are commonly used in Wikipedia
    
    # Loop through all the tables found and extract the column headers (th)
    for table in tables:
        # Find all header cells (th) in the first row of the table
        header_cells = table.find_all('th')
        
        # Extract and print the text from each header cell
        column_names = [cell.get_text(strip=True) for cell in header_cells]
        
        # Print the column names
        print("Column Names in the Table:")
        print(column_names)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Column Names in the Table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Column Names in the Table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '8', '9', '10', '11', '12', '13']
Column Names in the Table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
Column Names in the Table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '21', '22', '23', '24', '25', '26', '27', '28', 'N/A[e]']
Column Names in the Table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]

Starting from the third table is our target table contains the actual launch records.


In [22]:
# Let's print the third table and check its content
#first_launch_table = html_tables[2]
#print(first_launch_table)

import requests
from bs4 import BeautifulSoup



# Check if the request was successful
if response.status_code == 200:
    # Use BeautifulSoup() to create a BeautifulSoup object from the response text content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all the tables on the page with class 'wikitable'
    html_tables = soup.find_all('table', {'class': 'wikitable'})  # 'wikitable' class is used for tables in Wikipedia
    
    # Check if the third table exists and print its content
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]  # Access the third table (index 2)
        print(first_launch_table)  # Print the third table
    else:
        print("Less than 3 tables found on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-2"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-2"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [24]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
import requests
from bs4 import BeautifulSoup

# Define the URL for the Falcon 9 Launch Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request to fetch the page content
response = requests.get(static_url)

# Define the function to extract column names from header
def extract_column_from_header(header_tag):
    return header_tag.get_text(strip=True)  # Get the text content and remove leading/trailing spaces

# Check if the request was successful
if response.status_code == 200:
    # Use BeautifulSoup() to create a BeautifulSoup object from the response text content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all the tables on the page with class 'wikitable'
    html_tables = soup.find_all('table', {'class': 'wikitable'})  # 'wikitable' class is used for tables in Wikipedia
    
    # Check if the third table exists and extract columns
    if len(html_tables) >= 3:
        first_launch_table = html_tables[2]  # Access the third table (index 2)
        
        # Find all header cells (th) in the first row of the table
        header_cells = first_launch_table.find_all('th')
        
        # Extract and print column names using the extract_column_from_header function
        column_names = [extract_column_from_header(cell) for cell in header_cells]
        
        # Print the extracted column names
        print("Extracted Column Names:")
        print(column_names)
    else:
        print("Less than 3 tables found on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



Extracted Column Names:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']


Check the extracted column names


In [26]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [30]:
# Initialize launch_dict with column names as keys and empty lists as values
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column from the dictionary (ensure the column name matches exactly)
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Initialize the dictionary with additional necessary columns and empty lists
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add new columns that may not be present in the initial extraction
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Print launch_dict to verify the structure
print(launch_dict)


{'Flight No.': [], 'Date andtime (UTC)': None, 'Version,Booster[b]': None, 'Launch site': [], 'Payload[c]': None, 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launchoutcome': None, 'Boosterlanding': None, '14': None, '15': None, '16': None, '17': None, '18': None, '19': None, '20': None, 'Payload': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [42]:
# Initialize the extracted_row counter
extracted_row = 0

# Iterate through all tables with class 'wikitable plainrowheaders collapsible'
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Iterate through all table rows (tr)
    for rows in table.find_all("tr"):
        # Check if the first table header is a flight number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # Check if it's a valid number
            else:
                flag = False
        else:
            flag = False
        
        # Get all the table elements (cells) for the current row
        row = rows.find_all('td')
        
        # If the flight number is valid (i.e., it is a digit), process the row
        if flag and row:
            extracted_row += 1

            # Extract Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Extract Date and Time
            datatimelist = date_time(row[0])  # Assuming date_time() is a function you've defined
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Extract Booster Version (handle if it's missing)
            bv = booster_version(row[1])  # Assuming booster_version() is a function you've defined
            if not bv and row[1].a:
                bv = row[1].a.string  # Fallback if 'a' tag is available
            launch_dict['Version Booster'].append(bv)

            # Extract Launch Site
            launch_site = row[2].a.string if row[2].a else "Unknown"  # Handle missing link
            launch_dict['Launch site'].append(launch_site)

            # Extract Payload
            payload = row[3].a.string if row[3].a else "Unknown"  # Handle missing link
            launch_dict['Payload'].append(payload)

            # Extract Payload Mass
            payload_mass = get_mass(row[4])  # Assuming get_mass() is a function you've defined
            launch_dict['Payload mass'].append(payload_mass)

            # Extract Orbit
            orbit = row[5].a.string if row[5].a else "Unknown"  # Handle missing link
            launch_dict['Orbit'].append(orbit)

            # Extract Customer
            customer = row[6].a.string if row[6].a else "Unknown"  # Handle missing link
            launch_dict['Customer'].append(customer)

            # Extract Launch Outcome
            launch_outcome = list(row[7].strings)[0] if row[7] else "Unknown"  # Handle missing outcome
            launch_dict['Launch outcome'].append(launch_outcome)

            # Extract Booster Landing (handle missing value)
            booster_landing = landing_status(row[8])  # Assuming landing_status() is a function you've defined
            launch_dict['Booster landing'].append(booster_landing)

            # Optional: Print for debugging
            # print(f"Flight No: {flight_number}, Date: {date}, Time: {time}, Payload: {payload}, Outcome: {launch_outcome}")



            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [48]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
print (df)

    Flight No. Date andtime (UTC) Version,Booster[b] Launch site Payload[c]  \
0            1                NaN                NaN       CCAFS        NaN   
1            2                NaN                NaN       CCAFS        NaN   
2            3                NaN                NaN       CCAFS        NaN   
3            4                NaN                NaN       CCAFS        NaN   
4            5                NaN                NaN       CCAFS        NaN   
..         ...                ...                ...         ...        ...   
237        117                NaN                NaN       CCSFS        NaN   
238        118                NaN                NaN         KSC        NaN   
239        119                NaN                NaN       CCSFS        NaN   
240        120                NaN                NaN         KSC        NaN   
241        121                NaN                NaN       CCSFS        NaN   

    Payload mass Orbit   Customer Launchoutcome Boo

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [55]:

df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
